# Análisis de la TRM — Notebook para entrega del Taller Final
**Objetivo:** Analizar la Tasa Representativa del Mercado (TRM) con dos fuentes (diaria y histórica), limpiar los datos, generar métricas clave (volatilidad, retornos, promedios móviles) y exportar resultados listos para Power BI.

**Archivos en este entorno / referencias**
- Instrucciones y rúbrica: `/mnt/data/U_Sabana_Programacion_Taller_3.pdf` (archivo subido por el usuario).
- Salidas generadas por este notebook se guardarán en `/mnt/data/` (gráficos y CSVs).


In [ ]:

# 1) CONFIGURACIÓN DE FUENTES
# Coloca aquí los nombres de archivo locales (si ya descargaste los CSVs) O las URLs.
# El notebook intentará primero abrir los archivos locales y si no los encuentra, intentará descargar desde las URLs.

trm_daily_local = "TRM_daily.csv"        # <-- si ya lo descargaste, pon el nombre correcto aquí
trm_hist_local  = "TRM_historic.csv"     # <-- si ya lo descargaste, pon el nombre correcto aquí

# Si prefieres leer directamente desde la web, reemplaza las URL abajo.
trm_daily_url = "https://www.datos.gov.co/api/views/mcec-87by/rows.csv?accessType=DOWNLOAD"
trm_hist_url  = "https://www.datos.gov.co/api/views/32sa-8pi3/rows.csv?accessType=DOWNLOAD"

print("Revisa las variables trm_daily_local / trm_hist_local y las URLs antes de ejecutar las celdas.")


In [ ]:

# 2) IMPORTS
import os
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt

# Para evitar problemas visuales en Power BI export, guardamos figuras como PNG en /mnt/data
out_dir = "/mnt/data"
os.makedirs(out_dir, exist_ok=True)


In [ ]:

# 3) CARGA DE DATOS (intenta local -> luego web)
def try_read(path, url=None, parse_dates=['date']):
    if os.path.exists(path):
        print(f"Cargando local: {path}")
        df = pd.read_csv(path, parse_dates=parse_dates, dayfirst=False, low_memory=False)
        return df
    elif url:
        print(f"No se encontró {path}. Intentando descargar desde URL...")
        df = pd.read_csv(url, parse_dates=parse_dates, dayfirst=False, low_memory=False)
        return df
    else:
        raise FileNotFoundError(f"No se encontró {path} y no hay URL para descargar.")

# Ajusta los nombres de columna `date_col` según tu CSV si es necesario.
date_col = None  # si tus archivos ya traen columna 'date', déjalo None. Si no, pon el nombre, por ejemplo 'Fecha'.

# Intento de lectura
try:
    trm_daily = try_read(trm_daily_local, trm_daily_url, parse_dates=[0])
    print("TRM diaria cargada. Shape:", trm_daily.shape)
except Exception as e:
    print("Error cargando TRM diaria:", e)
    trm_daily = None

try:
    trm_hist = try_read(trm_hist_local, trm_hist_url, parse_dates=[0])
    print("TRM histórica cargada. Shape:", trm_hist.shape)
except Exception as e:
    print("Error cargando TRM histórica:", e)
    trm_hist = None

# Mostrar primeras filas si se cargaron
if trm_daily is not None:
    display(trm_daily.head())
if trm_hist is not None:
    display(trm_hist.head())


In [ ]:

# 4) LIMPIEZA Y ESTANDARIZACIÓN
def standardize_trm_df(df, value_cols=None):
    df = df.copy()
    # Detectar columna fecha
    # Heurística: la primera columna de tipo datetime, o columna que contenga 'date' o 'fecha'
    date_cols = [c for c in df.columns if 'date' in c.lower() or 'fecha' in c.lower()]
    if not date_cols:
        # buscar la primera columna tipo object que parezca fecha
        for c in df.columns:
            try:
                df[c] = pd.to_datetime(df[c])
                date_cols = [c]
                break
            except:
                continue
    date_col = date_cols[0]
    df = df.rename(columns={date_col: 'date'})
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values('date').drop_duplicates(subset=['date'])
    # Buscar columna de valor (TRM)
    if value_cols:
        val_col = [c for c in df.columns if c in value_cols][0]
    else:
        # heurística: columna numérica más probable
        numeric_cols = df.select_dtypes(include=['number']).columns.tolist()
        if len(numeric_cols)==1:
            val_col = numeric_cols[0]
        else:
            # intenta columnas con 'trm' o 'valor' en el nombre
            candidates = [c for c in df.columns if 'trm' in c.lower() or 'valor' in c.lower() or 'rate' in c.lower()]
            val_col = candidates[0] if candidates else numeric_cols[0]
    df = df.rename(columns={val_col: 'trm'})
    # Limpieza de trm: eliminar comas, signos, convertir a float
    df['trm'] = df['trm'].astype(str).str.replace(',', '').str.replace('$', '').str.strip()
    df['trm'] = pd.to_numeric(df['trm'], errors='coerce')
    df = df.dropna(subset=['trm'])
    return df[['date', 'trm']]

if trm_daily is not None:
    trm_daily_clean = standardize_trm_df(trm_daily)
    print("TRM diaria limpia:", trm_daily_clean.shape)
    display(trm_daily_clean.head())

if trm_hist is not None:
    trm_hist_clean = standardize_trm_df(trm_hist)
    print("TRM histórica limpia:", trm_hist_clean.shape)
    display(trm_hist_clean.head())


In [ ]:

# 5) FEATURE ENGINEERING
# Unir ambos dataframes por fecha. Si hay solapamiento, preferimos el más granular (daily).
if 'trm_daily_clean' in globals() and 'trm_hist_clean' in globals():
    # Combine: take union, prefer daily for overlapping dates
    combined = pd.concat([trm_hist_clean, trm_daily_clean]).drop_duplicates(subset=['date'], keep='last')
elif 'trm_daily_clean' in globals():
    combined = trm_daily_clean.copy()
elif 'trm_hist_clean' in globals():
    combined = trm_hist_clean.copy()
else:
    combined = pd.DataFrame(columns=['date','trm'])

combined = combined.sort_values('date').reset_index(drop=True)
combined['trm_shift1'] = combined['trm'].shift(1)
combined['trm_change'] = combined['trm'] - combined['trm_shift1']
combined['trm_pct_change'] = combined['trm'].pct_change() * 100
combined['rolling_30'] = combined['trm'].rolling(window=30, min_periods=7).mean()
combined['rolling_90'] = combined['trm'].rolling(window=90, min_periods=14).mean()
combined['vol_30'] = combined['trm_pct_change'].rolling(window=30).std()  # volatilidad (pct) en ventana 30
combined['year'] = combined['date'].dt.year
combined['month'] = combined['date'].dt.month

print("Combined shape:", combined.shape)
display(combined.head())


In [ ]:

# 6) ANÁLISIS RÁPIDO Y GRÁFICOS (guardados en /mnt/data)
if combined.shape[0]==0:
    print("No hay datos en 'combined' — verifica las lecturas anteriores.")
else:
    # KPI básicos
    latest = combined.dropna().iloc[-1]
    kpis = {
        'TRM_última': latest['trm'],
        'Cambio_pct_último_día': latest['trm_pct_change'],
        'Volatilidad_últimos_30d_pct': combined['vol_30'].dropna().iloc[-1] if combined['vol_30'].dropna().shape[0] else np.nan
    }
    print("KPIs:", kpis)

    # Serie de tiempo: TRM
    plt.figure(figsize=(10,4))
    plt.plot(combined['date'], combined['trm'])
    plt.title("TRM — Serie de tiempo")
    plt.xlabel("Fecha")
    plt.ylabel("TRM")
    fn = os.path.join(out_dir, "trm_timeseries.png")
    plt.tight_layout()
    plt.savefig(fn)
    plt.close()
    print("Guardado:", fn)

    # Rolling means
    plt.figure(figsize=(10,4))
    plt.plot(combined['date'], combined['trm'], label='TRM')
    plt.plot(combined['date'], combined['rolling_30'], label='MA 30')
    plt.plot(combined['date'], combined['rolling_90'], label='MA 90')
    plt.legend()
    plt.title("TRM y promedios móviles")
    fn = os.path.join(out_dir, "trm_rolling.png")
    plt.tight_layout()
    plt.savefig(fn)
    plt.close()
    print("Guardado:", fn)

    # Volatilidad (30d)
    plt.figure(figsize=(10,4))
    plt.plot(combined['date'], combined['vol_30'])
    plt.title("Volatilidad TRM (std % cambio, ventana 30 días)")
    plt.xlabel("Fecha")
    plt.ylabel("Volatilidad (%)")
    fn = os.path.join(out_dir, "trm_volatility_30d.png")
    plt.tight_layout()
    plt.savefig(fn)
    plt.close()
    print("Guardado:", fn)

    # Calendar heatmap-like: promedio por mes-año (tabla pivot)
    pivot = combined.pivot_table(values='trm', index='year', columns='month', aggfunc='mean')
    pivot.to_csv(os.path.join(out_dir, "trm_pivot_year_month.csv"))
    print("Pivot guardado:", os.path.join(out_dir, "trm_pivot_year_month.csv"))


In [ ]:

# 7) EXPORTACIÓN PARA POWER BI Y ENTREGA
cleaned_csv = os.path.join(out_dir, "trm_combined_cleaned.csv")
combined.to_csv(cleaned_csv, index=False)
print("CSV limpio exportado para Power BI:", cleaned_csv)

# Guardar también un resumen anual con métricas útiles
annual = combined.groupby('year').agg({
    'trm': ['mean','min','max'],
    'trm_pct_change': 'std',
    'vol_30': 'mean'
}).reset_index()
annual.columns = ['year','trm_mean','trm_min','trm_max','trm_pct_change_std','vol30_mean']
annual_csv = os.path.join(out_dir, "trm_annual_summary.csv")
annual.to_csv(annual_csv, index=False)
print("Resumen anual exportado:", annual_csv)
display(annual.head())



# Próximos pasos para la entrega
1. Si no descargaste los CSVs locales, puedes ejecutar la celda que define `trm_daily_url` y `trm_hist_url` y el notebook intentará bajarlos.
2. Revisa las primeras celdas y ajusta `trm_daily_local` / `trm_hist_local` si tienes los archivos con otros nombres.
3. Ejecuta todas las celdas (orden superior -> inferior). El notebook:
   - Generará gráficos en `/mnt/data/` (PNG)
   - Generará `trm_combined_cleaned.csv` listo para importar a Power BI
   - Generará `trm_annual_summary.csv` con métricas por año
4. En Power BI: importa `trm_combined_cleaned.csv`, crea relaciones por fecha (si agregas otras tablas luego) y monta las visualizaciones.
5. Rúbrica: usa los gráficos generados como soporte para las 5 conclusiones que se incluirán en la página "Conclusiones Clave".

**Archivo de referencia del curso (rúbrica e instrucciones):** `/mnt/data/U_Sabana_Programacion_Taller_3.pdf`
